# Get Data by Metro Area

## Load Packages

In [14]:
# Import packages to use later
import pandas as pd
import nltk
import time
import json

## Investigate Cities

In [15]:
businesses = pd.read_csv("yelp_academic_dataset_business.csv")
businesses = businesses.rename(columns={'business': 'business_id'})

d = dict(businesses['metroarea'].value_counts())

for city, count in sorted(d.items(), key=lambda x: x[1], reverse=True):
    if count > 1000:
        print(city, count)

Philadelphia 14569
Tucson 9250
Tampa 9050
Indianapolis 7540
Nashville 6971


In [16]:
businesses.head(3)

,city,review_count,name,business_id,longitude,state,stars,address,latitude,metroarea,categories
0,Tucson,22,Target,tUFrWirKiKi_TAnsVWINQQ,-110.880452,AZ,3.5,5255 E Broadway Blvd,32.223236,Tucson,"Department Stores, Shopping, Fashion, Home & G..."
1,Philadelphia,80,St Honore Pastries,MTSW4McQd7CbVtyjqoe9mw,-75.155564,PA,4.0,935 Race St,39.955505,Philadelphia,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
2,Nashville,10,Sonic Drive-In,bBDDEgkFA1Otx9Lfe7BZUQ,-86.768170,TN,1.5,2312 Dickerson Pike,36.208102,Nashville,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."


## Dataset for Each Metro

In [17]:
# These cities contain almost all reviews in the dataset (removing outlier entries)
metros = [city for city, count in sorted(d.items(), key=lambda x: x[1], reverse=True)]
print(metros)

# Initialize empty dataframe for each city
metro_reviews = {metro:pd.DataFrame() for metro in metros}

# Dictionary of city IDs to set of business IDs in that city
metro_businesses = {metro:set(businesses[businesses['metroarea'] == metro]['business_id'].tolist()) for metro in metros}
print(metro_businesses)

num_total_reviews = 0

with pd.read_csv("yelp_academic_dataset_review.csv", chunksize=100000) as reader:
    for count, chunk in enumerate(reader):
        #####
        chunk['datetime'] = pd.to_datetime(chunk['date'])
        chunk = chunk[chunk['datetime'] >= pd.Timestamp('2010-01-01')]
        chunk = chunk[chunk['datetime'] < pd.Timestamp('2020-01-01')]
        #print(chunk.head(5))
        #####
        num_total_reviews += len(chunk)
        for metro in metros:
            vals = chunk[chunk.business_id.isin(metro_businesses[metro])]
            metro_reviews[metro] = pd.concat([metro_reviews[metro], vals], axis=0)
        if count % 10 == 0:
            print("Chunk #", count)
        #break
    print("End: Chunk #", count)

print(num_total_reviews)
    
for metro in metros:
    #metro_reviews[metro].to_csv("small_reviews/yelp_academic_dataset_reviews_" + metro + ".csv")
    metro_reviews[metro].to_csv("small_reviews_urbcomp/yelp_academic_dataset_reviews_" + metro + ".csv")
    print(metro_reviews[metro].head(5))
    # business_review_count = metro_reviews[metro].groupby(
    #     'business_id')['review_count'].agg(['median', 'sum'])


['Philadelphia', 'Tucson', 'Tampa', 'Indianapolis', 'Nashville']
{'Philadelphia': {'9o55tW2eCwxRvwWVQSzj5g', 'uI5k9lJkWlKi6OUW5yYIpQ', '7gI3LKutnsliBXV9OCuyGw', 'k2LRL37Ubi9DcHbhsIeUbA', 'klNW30cZ_idEMGWgT9pLvA', '_SUBeKkT-JwivJtC6f1Etw', '1DwfadxGPydTonvpeU68Mw', 'EplLH353uir93Jo_Py3xww', 'PmAd6HbeqJLMy0s9txsZzw', 'bzgAFmNxvQ7i-YscAiDoDA', 'uVri4t3oyA7gqbX5q7_0LA', 'YzluF3FNGjnW-tOrLZt8JA', 'DGY_XF6q0_-I_zQoeewlMQ', 'viC-j3vKaSl2rUuR0gFWNQ', 'OHzX-ZD9qyoeoxR8Z0dlIA', '-tQ0lqgfY8i5avBVyUMYWw', 's55Hw6gHhQzZctinFhPu-g', 'wVuY_KHHmKsXKIkZeiqIFg', 'WKTrjBhrx_SPZS7JVZnzFQ', 'VZKgw_0qDGWFkZKjG2tTrQ', 'BzDbphyIfHWZUoaQQdAUYw', 'H_JO3CNWRTRHmstEyvb31Q', 'olp3UKVlryO9x1e-egn8aA', 'b66R96v0N-Mmu0jb4cVhiQ', 'uNh3Eeucqrd07qJ8hG11dQ', '7HcaaocVkMj3g7OiIVWy6g', 'gh2Oc17i-FYsUYU1kcmP5w', 'acsheIhuYrmmth1_z0Ylxw', 'V5KiEoH4E86GA2S3RQkf1g', '1-zkyTwnpeZn9MmBpGSrqQ', 'S9JyjLuKyyHB8a_2rkyN1Q', '7xlElnqpIR2NuwUOpZLM6Q', 'D0gEvEMLeag7CtcaCCStPg', 'khn9lsIDvoULtuGP0zYaHw', 'eznriqn0sKt0D594H5OCcA', 'FHinXL

In [24]:
import plotly.subplots as sp
import plotly.express as px

fig = sp.make_subplots(rows=1, cols=len(metros), subplot_titles=[
                       f"{metro}" for metro in metros], shared_yaxes=True)

for i, metro in enumerate(metros):
    df = metro_reviews[metro]
    review_counts = df.groupby('business_id')['review_id'].count()
    fig.add_box(x=review_counts, row=1, col=i+1)
    fig.update_xaxes(title_text='Review Count', row=1, col=i+1)
    fig.update_yaxes(title_text='Metro', row=1, col=i+1)

fig.show()


## Checking Results

In [19]:
print(metros)

['Philadelphia', 'Tucson', 'Tampa', 'Indianapolis', 'Nashville']


In [20]:
lengths = [len(metro_reviews[metro]) for metro in metros]
print(lengths)
print(sum(lengths))
print(num_total_reviews)

[813634, 321269, 359167, 289986, 361162]
2145218
5643186
